In [2]:
import pandas as pd
import numpy as np

## Ensembl

See [plugins/gene/ensembl.py](https://github.com/opentargets/data_pipeline/blob/329ff219f9510d137c7609478b05d358c9195579/mrtarget/plugins/gene/ensembl.py)

In [23]:
url = 'https://storage.googleapis.com/open-targets-data-releases/19.11/input/annotation-files/homo_sapiens_core_98_38_genes.json.gz'
df = pd.read_json(url, lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60564 entries, 0 to 60563
Data columns (total 17 columns):
id                 60564 non-null object
display_name       60564 non-null object
description        60564 non-null object
biotype            60564 non-null object
source             60564 non-null object
version            60564 non-null int64
logic_name         60564 non-null object
seq_region_name    60564 non-null object
start              60564 non-null int64
end                60564 non-null int64
strand             60564 non-null int64
assembly_name      60564 non-null object
db_type            60564 non-null object
ensembl_release    60564 non-null int64
species            60564 non-null object
object_type        60564 non-null object
is_reference       60564 non-null bool
dtypes: bool(1), int64(5), object(11)
memory usage: 7.5+ MB


In [13]:
df.head(3)

,id,display_name,description,biotype,source,version,logic_name,seq_region_name,start,end,strand,assembly_name,db_type,ensembl_release,species,object_type,is_reference
0,ENSG00000210049,MT-TF,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...,Mt_tRNA,insdc,1,mt_genbank_import_homo_sapiens,MT,577,647,1,GRCh38,core,98,homo_sapiens,Gene,True
1,ENSG00000211459,MT-RNR1,mitochondrially encoded 12S rRNA [Source:HGNC ...,Mt_rRNA,insdc,2,mt_genbank_import_homo_sapiens,MT,648,1601,1,GRCh38,core,98,homo_sapiens,Gene,True
2,ENSG00000210077,MT-TV,mitochondrially encoded tRNA-Val (GUN) [Source...,Mt_tRNA,insdc,1,mt_genbank_import_homo_sapiens,MT,1602,1670,1,GRCh38,core,98,homo_sapiens,Gene,True


In [24]:
df.id.is_unique

True

In [18]:
df.assembly_name.value_counts()

GRCh38    60564
Name: assembly_name, dtype: int64

In [19]:
df.seq_region_name.value_counts() 

1     5472
2     4197
11    3361
3     3185
17    3060
6     3057
12    3052
7     3014
19    2992
5     2984
4     2653
16    2555
8     2482
X     2423
10    2332
9     2326
14    2286
15    2219
20    1457
13    1398
22    1386
18    1242
21     872
Y      522
MT      37
Name: seq_region_name, dtype: int64

In [17]:
df.ensembl_release.value_counts()

98    60564
Name: ensembl_release, dtype: int64

In [21]:
df.id.str.len().value_counts()

15    60564
Name: id, dtype: int64

### HGNC

See [plugins/gene/hgnc.py](https://github.com/opentargets/data_pipeline/blob/329ff219f9510d137c7609478b05d358c9195579/mrtarget/plugins/gene/hgnc.py)

In [30]:
!wget -P /tmp https://storage.googleapis.com/open-targets-data-releases/19.11/input/annotation-files/hgnc_complete_set-2019-10-31.json

--2019-12-20 16:27:28--  https://storage.googleapis.com/open-targets-data-releases/19.11/input/annotation-files/hgnc_complete_set-2019-10-31.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.240, 2607:f8b0:4004:808::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31913288 (30M) [application/json]
Saving to: ‘/tmp/hgnc_complete_set-2019-10-31.json’

hgnc_complete_set-2 100%[===================>]  30.43M  8.54MB/s    in 3.9s    

2019-12-20 16:27:33 (7.84 MB/s) - ‘/tmp/hgnc_complete_set-2019-10-31.json’ saved [31913288/31913288]



In [49]:
import json
with open('/tmp/hgnc_complete_set-2019-10-31.json', 'r') as fd:
    # Immediately filter to ensembl_gene presence as all others are ignored in module
    df = pd.DataFrame(json.load(fd)['response']['docs'])\
        .pipe(lambda df: df[df['ensembl_gene_id'].notnull()])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38385 entries, 0 to 43378
Data columns (total 52 columns):
date_approved_reserved    38385 non-null object
vega_id                   32076 non-null object
locus_group               38385 non-null object
status                    38385 non-null object
_version_                 38385 non-null int64
uuid                      38385 non-null object
merops                    716 non-null object
refseq_accession          37798 non-null object
locus_type                38385 non-null object
cosmic                    17479 non-null object
hgnc_id                   38385 non-null object
rgd_id                    17928 non-null object
ensembl_gene_id           38385 non-null object
entrez_id                 38376 non-null object
gene_group                18684 non-null object
omim_id                   15959 non-null object
symbol                    38385 non-null object
location                  38367 non-null object
name                      3838

In [50]:
(df.ensembl_gene_id.dropna().str.len() == 0).sum()

0

In [51]:
df.ensembl_gene_id.is_unique, df.hgnc_id.is_unique

(False, True)

In [52]:
df.groupby(['ensembl_gene_id', 'hgnc_id']).size().value_counts()

1    38385
dtype: int64

In [53]:
df['status'].value_counts()

Approved    38385
Name: status, dtype: int64